In [1]:
# Establish connections to the input db and variables spreadsheets

# Import modules
import os, re, math
import arcpy
import pandas as pd
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from dotenv import load_dotenv

# Lot the .env variables
load_dotenv()

# Set the data paths
inputs_path = os.getenv('J111_INPUTS')
data_path = os.getenv('J122_DATA')
spatial_path = os.getenv('J111_SPATIAL')
env_path = os.getenv('J111_ENV')

# Get full path of Input GDBs
in_gdb_path = os.path.join(spatial_path, 'redland_gam_alloc.gdb')
out_gdb_path = os.path.join(spatial_path, 'redland_gam_idm.gdb')

# Set arcpy workspace
arcpy.env.workspace = os.path.join(env_path, 'J111_redland_gam.gdb')

# Inspect
in_gdb_path

'G:\\\\Shared drives\\\\PIESolutions_03_Projects\\\\J000111 - Redlands planning assumption update\\\\06_Working Documents\\\\00_GIS Directory\\\\00_Data\\\\gam_model_run\\redland_gam_alloc.gdb'

In [2]:
''' 
----------------------------------
Import and prepare the input datasets
----------------------------------
'''

# Import the residential and non-residential development projections
res_dp = pd.DataFrame.spatial.from_featureclass(os.path.join(in_gdb_path, 'OUT_FC_PropertyBase_ResGrowthAllocation')).drop(['OBJECTID'], axis=1)
nonres_dp = pd.DataFrame.spatial.from_featureclass(os.path.join(in_gdb_path, 'OUT_FC_PropertyBase_NonResGrowthAllocation')).drop(['OBJECTID', 'SHAPE', 'sa2_name'], axis=1)

dp = pd.merge(left=res_dp, right=nonres_dp, on='pbno')

# Import current water consumption feature class
wc = pd.DataFrame.spatial.from_featureclass(os.path.join(out_gdb_path, 'OUT_FC_PropertyBase_WaterConsumption')).drop(['OBJECTID', 'SHAPE', 'landno', 'propno', 'segpar', 'prop_use'], axis=1)

# Inspect
dp.head()

,pbno,sa2_name,det_dwl_2021,det_dwl_2026,det_dwl_2031,det_dwl_2036,det_dwl_2041,det_dwl_2046,det_dwl_2051,det_dwl_spare,...,other_cap,retail_2021,retail_2026,retail_2031,retail_2036,retail_2041,retail_2046,retail_2051,retail_spare,retail_cap
0,8000000,Cleveland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8000001,Cleveland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8000002,Redland Islands,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,672.848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8000003,Cleveland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8000004,Capalaba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
''' 
----------------------------------
Get the contribution of existing non projected development as an ep
----------------------------------
'''
wc_nodev = wc.loc[:, ('pbno', 'nodev_wc')]

wc_nodev['ep_2021'] = ((wc_nodev['nodev_wc'] * 1000) / 365) / 230

wc_nodev = wc_nodev.drop(['nodev_wc'], axis=1).set_index('pbno').iloc[:,0]

wc_nodev

pbno
8000000    0.0
8000001    0.0
8000002    0.0
8000003    0.0
8000004    0.0
          ... 
8068557    0.0
8068558    0.0
8068559    0.0
8068560    0.0
8068561    0.0
Name: ep_2021, Length: 68500, dtype: float64

In [4]:
ep_rates = {
    "att_dwl" : 1.7,
    "det_dwl" : 2.7,
    "commerical" : 0.004,
    "community" : 0.006,
    "education" : 0.005,
    "health" : 0.012,
    "industrial" : 0.010,
    "other" : 0.006,
    "retail" : 0.005
}

ep_years = {
    '2021' : 'ep_2021', 
    '2026' : 'ep_2026', 
    '2031' : 'ep_2031' , 
    '2036' : 'ep_2036', 
    '2041' : 'ep_2041', 
    '2046' : 'ep_2046', 
    '2051' : 'ep_2051',
    'spare' : 'ep_ult'
}

dpep = dp.copy()

for suffix, name in ep_years.items():

    dpep[f'{name}'] = 0.0

for index, row in dpep.iterrows():

    for suffix, col_name in ep_years.items():

        ep = 0.0
        ep_nodev = wc_nodev[dpep.iloc[index, :]['pbno']]

        for col, val in row.iteritems():

            if col.endswith(suffix):

                for dev, rate in ep_rates.items():

                    if col.startswith(dev):

                        ep += round(val * rate, 2)

        if suffix == 'spare':

            ep += dpep.iloc[index, :]['ep_2051']

        ep += ep_nodev

        dpep.at[index, col_name] = ep

dpep = dpep.loc[:, ('pbno', 'sa2_name', *[name for suffix, name in ep_years.items()])]

dpep.head()

,pbno,sa2_name,ep_2021,ep_2026,ep_2031,ep_2036,ep_2041,ep_2046,ep_2051,ep_ult
0,8000000,Cleveland,24.3,24.3,35.1,35.1,35.1,45.90,59.40,59.40
1,8000001,Cleveland,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
2,8000002,Redland Islands,48.6,48.6,48.6,48.6,48.6,50.18,52.64,52.64
3,8000003,Cleveland,394.2,394.2,394.2,394.2,394.2,394.20,394.20,394.20
4,8000004,Capalaba,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00


In [5]:
# Import the water zone details per property 

wsz = pd.DataFrame.spatial.from_table(os.path.join(out_gdb_path, 'INPUT_TB_Propbase_WaterSupplyZones')).drop(['OBJECTID'], axis=1)

wsz = wsz.loc[:, ('pbno', 'zone', 'prop_area', 'part_area')]

wsz = wsz.groupby(['pbno', 'zone', 'prop_area']).sum().reset_index()

wsz['portion'] = wsz.apply(lambda row: 1.0 if row['part_area'] == 0.0 else round(row['part_area'] / row['prop_area'], 2), axis=1)

wsz = wsz.loc[:, ('pbno', 'zone', 'portion')].drop_duplicates(subset=['pbno', 'zone', 'portion']).reset_index(drop=True)

wsz.head()

,pbno,zone,portion
0,8000000,FL_W004,1.0
1,8000001,FL_W004,1.0
2,8000002,FL_W014,1.0
3,8000003,FL_W004,1.0
4,8000004,FL_W004,1.0


In [6]:
# Join ep demand to water supply zone properties

wsz_ep = pd.merge(left=wsz, right=dpep, on='pbno')

for suffix, col_name in ep_years.items():

    wsz_ep[f'{col_name}'] = round(wsz_ep[f'{col_name}'] * wsz_ep['portion'], 2)

wsz_ep = wsz_ep.loc[:, ('zone', *[name for suffix, name in ep_years.items()])].groupby('zone').sum()

wsz_ep.head()

,ep_2021,ep_2026,ep_2031,ep_2036,ep_2041,ep_2046,ep_2051,ep_ult
zone,,,,,,,,
FL_W001,4270.66,4608.53,5054.03,5367.39,5667.52,6079.19,6341.07,7310.50
FL_W002,1107.11,1115.75,1131.05,1139.55,1139.55,1139.55,1139.55,1140.10
FL_W003,866.17,866.17,930.54,971.21,971.21,971.21,971.21,1026.35
FL_W004,68078.07,74720.82,81295.67,85664.86,89862.01,91909.42,93795.92,115116.43
FL_W005,8114.37,8254.36,8342.44,8376.44,8376.44,8376.44,8376.44,10774.02


In [7]:
# Import the sewer zone details per property

sz = pd.DataFrame.spatial.from_table(os.path.join(out_gdb_path, 'INPUT_TB_Propbase_SewerZones')).drop(['OBJECTID'], axis=1)

sz = sz.loc[:, ('pbno', 'zone', 'prop_area', 'part_area')]

sz = sz.groupby(['pbno', 'zone', 'prop_area']).sum().reset_index()

sz['portion'] = sz.apply(lambda row: 1.0 if row['part_area'] == 0.0 else round(row['part_area'] / row['prop_area'], 2), axis=1)

sz = sz.loc[:, ('pbno', 'zone', 'portion')].drop_duplicates(subset=['pbno', 'zone', 'portion']).set_index('pbno')

sz.head()

,zone,portion
pbno,,
8000000,S001,1.0
8000001,S005,1.0
8000002,S131,1.0
8000003,S074,1.0
8000004,S065,1.0


In [8]:
# Join ep demand to sewer supply zone properties

sz_ep = pd.merge(left=sz, right=dpep, on='pbno')

for suffix, col_name in ep_years.items():

    sz_ep[f'{col_name}'] = round(sz_ep[f'{col_name}'] * sz_ep['portion'], 2)

sz_ep = sz_ep.loc[:, ('zone', *[name for suffix, name in ep_years.items()])].groupby('zone').sum()

sz_ep.head()

,ep_2021,ep_2026,ep_2031,ep_2036,ep_2041,ep_2046,ep_2051,ep_ult
zone,,,,,,,,
S158,241.49,241.49,241.49,241.49,241.49,241.49,241.49,241.59
F201,1042.46,1211.64,1222.44,1238.64,1238.64,1238.64,1238.64,1516.97
F202,930.53,839.65,839.65,839.65,839.65,839.65,839.65,880.18
F203,382.95,385.65,391.05,399.15,399.15,399.15,399.15,507.75
F204,447.86,491.70,620.75,794.48,848.28,886.42,925.01,1916.55


In [9]:
# Get the ep demand per sa2

abs_ep = dpep.drop(['pbno'], axis=1).groupby('sa2_name').sum()

abs_ep

,ep_2021,ep_2026,ep_2031,ep_2036,ep_2041,ep_2046,ep_2051,ep_ult
sa2_name,,,,,,,,
Alexandra Hills,12212.443389,12478.463389,12590.663389,12641.663389,12704.133389,12710.903389,12718.213389,14860.936778
Birkdale,11429.602073,11710.602073,12066.102073,12397.002073,12727.902073,12817.402073,12906.902073,16218.054145
Capalaba,19972.139649,21159.909649,22837.709649,23473.259649,23900.439649,24520.929649,25018.949649,35524.429297
Cleveland,19465.685747,24031.455747,28469.255747,31903.815747,35306.245747,36837.915747,38301.295747,42556.111495
Ormiston,5845.004628,6143.074628,6505.454628,6694.604628,6831.144628,6863.404628,6887.704628,7575.069256
Redland Bay,12847.087802,14144.107802,16021.267802,17555.747802,19430.037802,20045.687802,20536.997802,28038.285605
Redland Islands,14266.432615,15341.462615,16482.702615,17335.672615,17659.102615,17703.292615,17746.842615,21567.525229
Sheldon - Mount Cotton,5852.602984,5897.692984,5957.492984,5976.492984,5981.892984,5987.292984,5992.692984,6146.175968
Thorneside,3562.425902,3565.525902,3619.145902,3660.405902,3713.165902,3832.145902,3859.035902,4924.641805


In [54]:
''' 
----------------------------------
Output EP summary tables
----------------------------------
'''

outputs_path = os.path.join(data_path, 'idm_outputs')

# Output metrics to csv
wsz_ep.to_csv(os.path.join(outputs_path, 'OUT_RedlandWater_IDM_EPDemandByWaterZone.csv'))
sz_ep.to_csv(os.path.join(outputs_path, 'OUT_RedlandWater_IDM_EPDemandBySewerZone.csv'))
abs_ep.to_csv(os.path.join(outputs_path, 'OUT_RedlandWater_IDM_EPDemandBySA2.csv'))

In [14]:
''' 
----------------------------------
Output EP Projections per Property
----------------------------------
'''

# Construt output featureclass
out_fc = pd.merge(left=res_dp.loc[:, ('pbno', 'SHAPE')], right=dpep, on='pbno')

# Output into geodatabase
out_fc.spatial.to_featureclass(os.path.join(out_gdb_path, 'OUT_FC_PropertyBase_EPDemand'), overwrite=True)

'G:\\Shared drives\\PIESolutions_03_Projects\\J000111 - Redlands planning assumption update\\06_Working Documents\\00_GIS Directory\\00_Data\\gam_model_run\\redland_gam_idm.gdb\\OUT_FC_PropertyBase_EPDemand'